In [ ]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

from skimage.color import rgb2gray, gray2rgb

Using TensorFlow backend.


In [ ]:
def load_data():
        color = []
        bw = []
        paths = []
        #get all files in this folder
        for r, d, f in os.walk(r"D:\Downloads\painter-by-numbers-resized"):
            for file in f:
                if '.jpg' in file:
                    paths.append(os.path.join(r, file))
        #for each file add normal resolution and small resolution to arrays
        for path in paths:
            img = Image.open(path)
            x = img.resize((128,128))
            placeholder = np.array(x)
            color.append(placeholder)
            bw.append(rgb2gray(placeholder))
            
        del paths    
       
        y_train = np.array(color)
        y_train = y_train.reshape(len(color),128,128,3)
        x_train = np.array(bw)
        x_train = x_train.reshape(len(bw),128,128,1)
        return y_train, x_train

In [3]:
class SRGAN():
    def __init__(self):
        
        # Shape of high resolution output image
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        # Shape of low resolution input image
        self.latent_dim = (128,128,1)

        #optimizer (learning rate and beta values)
        optimizer = Adam(0.0001, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=self.latent_dim)
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Conv2D(140, input_shape=self.latent_dim, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(140, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(140, kernel_size=(2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(140, kernel_size=(4,4), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(140, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(140, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(3, kernel_size=(2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.summary()

        noise = Input(shape=self.latent_dim)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        Y_train, X_train = load_data()

        # Rescale to be between 0 & 1
        X_train = X_train / 255
        Y_train = Y_train / 255

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Placeholder for loss function values
        g_loss_epochs = np.zeros((epochs, 1))
        d_loss_epochs = np.zeros((epochs, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, Y_train.shape[0], batch_size)
            imgs = Y_train[idx]

            # Generate super resolution images from the random batch of images
            gen_imgs = self.generator.predict(X_train[idx])

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(X_train[idx], valid)
            
            #save loss history
            g_loss_epochs[epoch] = g_loss
            d_loss_epochs[epoch] = d_loss[0]

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch, X_train, idx)
                
        return g_loss_epochs, d_loss_epochs

    def save_imgs(self, epoch, X_train, idx):
        r, c = 3, 3
        # Select 9 random images
        index = np.random.randint(0, X_train.shape[0], 9)
        images = X_train[idx]
        # Super resolution the images
        gen_imgs = self.generator.predict(images)
        gen_imgs = np.array(gen_imgs) * 255
        gen_imgs = gen_imgs.astype(int)
        # Plot each image
        fig=plt.figure(figsize=(20, 20))
        for i in range(1, c*r+1):
            img = gen_imgs[i-1]
            fig.add_subplot(r, c, i)
            plt.imshow(img)
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\epoch_%d.png" % epoch)
        plt.close()
        # save model to .h5 file
        self.generator.save("generator.h5")

In [4]:
gan = SRGAN()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 32, 64)       

In [5]:
g_loss, d_loss = gan.train(epochs=15001, batch_size=9, save_interval=100)

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.116400, acc.: 16.67%] [G loss: 0.472347]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 1.077887, acc.: 50.00%] [G loss: 0.673541]
2 [D loss: 0.911348, acc.: 50.00%] [G loss: 0.910210]
3 [D loss: 0.795538, acc.: 55.56%] [G loss: 0.752919]
4 [D loss: 0.766020, acc.: 50.00%] [G loss: 0.857400]
5 [D loss: 0.621384, acc.: 77.78%] [G loss: 0.693831]
6 [D loss: 0.490977, acc.: 77.78%] [G loss: 0.958575]
7 [D loss: 0.589162, acc.: 66.67%] [G loss: 0.917486]
8 [D loss: 0.647061, acc.: 66.67%] [G loss: 1.095881]
9 [D loss: 1.065752, acc.: 38.89%] [G loss: 1.539848]
10 [D loss: 0.549508, acc.: 61.11%] [G loss: 1.751008]
11 [D loss: 0.750736, acc.: 55.56%] [G loss: 1.589751]
12 [D loss: 0.681275, acc.: 66.67%] [G loss: 1.309301]
13 [D loss: 0.862175, acc.: 50.00%] [G loss: 1.204645]
14 [D loss: 0.689426, acc.: 72.22%] [G loss: 1.622092]
15 [D loss: 0.667622, acc.: 61.11%] [G loss: 1.502150]
16 [D loss: 1.045616, acc.: 50.00%] [G loss: 1.483633]
17 [D loss: 0.721022, acc.: 66.67%] [G loss: 1.171815]
18 [D loss: 1.136566, acc.: 27.78%] [G loss: 1.421440]
19 [D loss: 0.91177

150 [D loss: 0.630955, acc.: 72.22%] [G loss: 1.784110]
151 [D loss: 1.283080, acc.: 33.33%] [G loss: 0.903662]
152 [D loss: 0.844857, acc.: 61.11%] [G loss: 1.254593]
153 [D loss: 0.987293, acc.: 38.89%] [G loss: 1.435702]
154 [D loss: 1.586902, acc.: 16.67%] [G loss: 1.320295]
155 [D loss: 1.142019, acc.: 44.44%] [G loss: 1.388076]
156 [D loss: 0.979388, acc.: 50.00%] [G loss: 1.866286]
157 [D loss: 0.436321, acc.: 83.33%] [G loss: 1.939749]
158 [D loss: 0.726567, acc.: 61.11%] [G loss: 1.775325]
159 [D loss: 0.300394, acc.: 88.89%] [G loss: 2.262726]
160 [D loss: 0.505001, acc.: 77.78%] [G loss: 1.660451]
161 [D loss: 0.507514, acc.: 66.67%] [G loss: 1.948905]
162 [D loss: 0.566872, acc.: 66.67%] [G loss: 1.753202]
163 [D loss: 0.882512, acc.: 55.56%] [G loss: 2.350121]
164 [D loss: 0.556288, acc.: 61.11%] [G loss: 1.038687]
165 [D loss: 0.567698, acc.: 72.22%] [G loss: 1.661387]
166 [D loss: 1.163669, acc.: 27.78%] [G loss: 0.897035]
167 [D loss: 0.929224, acc.: 44.44%] [G loss: 1.

...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


3700 [D loss: 0.900035, acc.: 66.67%] [G loss: 3.700339]
3701 [D loss: 0.759618, acc.: 77.78%] [G loss: 3.743007]
3702 [D loss: 0.196926, acc.: 94.44%] [G loss: 4.205482]
3703 [D loss: 0.560345, acc.: 72.22%] [G loss: 1.782932]
3704 [D loss: 0.331051, acc.: 83.33%] [G loss: 3.537481]
3705 [D loss: 0.142546, acc.: 94.44%] [G loss: 4.405816]
3706 [D loss: 0.182799, acc.: 88.89%] [G loss: 4.140330]
3707 [D loss: 0.686518, acc.: 66.67%] [G loss: 3.458551]
3708 [D loss: 0.049612, acc.: 100.00%] [G loss: 5.204874]
3709 [D loss: 0.083866, acc.: 100.00%] [G loss: 5.252498]
3710 [D loss: 0.114759, acc.: 100.00%] [G loss: 4.262151]
3711 [D loss: 0.172241, acc.: 94.44%] [G loss: 3.801466]
3712 [D loss: 0.095387, acc.: 94.44%] [G loss: 7.761630]
3713 [D loss: 0.179120, acc.: 94.44%] [G loss: 5.573580]
3714 [D loss: 0.309821, acc.: 83.33%] [G loss: 3.798273]
3715 [D loss: 0.303493, acc.: 88.89%] [G loss: 4.342429]
3716 [D loss: 0.262244, acc.: 88.89%] [G loss: 5.330060]
3717 [D loss: 0.123139, acc.

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


3800 [D loss: 1.477979, acc.: 33.33%] [G loss: 2.634542]
3801 [D loss: 0.331854, acc.: 94.44%] [G loss: 4.398706]
3802 [D loss: 0.444461, acc.: 77.78%] [G loss: 3.658218]
3803 [D loss: 0.511189, acc.: 77.78%] [G loss: 4.332430]
3804 [D loss: 0.319891, acc.: 88.89%] [G loss: 4.388000]
3805 [D loss: 0.395834, acc.: 88.89%] [G loss: 4.443105]
3806 [D loss: 0.507496, acc.: 77.78%] [G loss: 2.936656]
3807 [D loss: 0.226869, acc.: 88.89%] [G loss: 3.118034]
3808 [D loss: 0.411245, acc.: 83.33%] [G loss: 3.199531]
3809 [D loss: 0.027725, acc.: 100.00%] [G loss: 7.316160]
3810 [D loss: 0.041191, acc.: 100.00%] [G loss: 5.908450]
3811 [D loss: 0.101973, acc.: 94.44%] [G loss: 5.325889]
3812 [D loss: 0.026226, acc.: 100.00%] [G loss: 5.014384]
3813 [D loss: 0.159411, acc.: 94.44%] [G loss: 5.427065]
3814 [D loss: 0.146637, acc.: 94.44%] [G loss: 4.462573]
3815 [D loss: 0.045159, acc.: 100.00%] [G loss: 5.098595]
3816 [D loss: 0.045252, acc.: 100.00%] [G loss: 4.118286]
3817 [D loss: 0.025341, ac

KeyboardInterrupt: 

In [ ]:
plt.plot(g_loss)
plt.plot(d_loss)
plt.title('GAN Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Generator', 'Discriminator'], loc='upper left')
plt.show()